In [ ]:


# first make sure eval works
## amount missing 
nonmiss = lcs.dataset[:,:,:,1].sum(axis=2) > 0
mis2 = (nonmiss.sum(axis=1) == 1)
mis1 = (nonmiss.sum(axis=1) == 2)
mis0 = (nonmiss.sum(axis=1) == 0)

misg = nonmiss[:,2]
misr = nonmiss[:,0]
misi = nonmiss[:,1]

# get anomoly predictions


# Subsampled Z more regular because thats what it sees during training?
# pca3d = pca.fit_transform(zs_sub.reshape(zs_sub.shape[0],-1)) 
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(pca3d[:,0],pca3d[:,1],pca3d[:,2])

# pl = pca3d[:,0:2].T
# k = gaussian_kde(pl, bw_method=0.1)(pl)
# fig = plt.figure(figsize=(12,10))
# ax = fig.add_subplot(111)
# dens = ax.scatter(pca3d[:,0],pca3d[:,1],c=k, alpha=0.4,s=80)
# fig.colorbar(dens,shrink=0.75)
# plt.savefig('../img/pca2d')

## don't look all that different
#zs_sub = (np.random.randn(n_samples, qzs_sub.shape[0], qzs_sub.shape[2],qzs_sub.shape[3]) * qzs_sub[:,1,:,:] + qzs_sub[:,0,:,:]).mean(0)



def evaluate_hetvae(
    net,
    dim,
    dataloader,
    frac=0.5,
    k_iwae=1,
    device='mps',
    forecast=False,
    qz_mean=False
):
    train_n = 0
    train_loss,avg_loglik, mse, mae = 0, 0, 0,0
    mean_mae, mean_mse = 0, 0
    individual_nlls = []
    indy_nlls = []
    mses= []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            # forecasting if this mask is set to first section of points only, not random selection
            subsampled_mask = make_masks(batch, frac=frac, forecast=forecast)
            ######################
            errorbars = torch.swapaxes(batch[:,:,:,2], 2,1)
            weights = errorbars.clone()
            weights[weights!=0] = 1 / weights[weights!=0]
            errorbars[errorbars!=0] = torch.log(errorbars[errorbars!=0])
            logerr = errorbars.to(device)
            weights = weights.to(device)
            ######################
            batch = batch.to(device)
            subsampled_mask = subsampled_mask.to(device)
            recon_mask = torch.logical_xor(subsampled_mask, batch[:,:,:,1])

            context_y = torch.cat((
              batch[:,:,:,1] * subsampled_mask, subsampled_mask
            ), 1).transpose(2,1)
            recon_context_y = torch.cat((
              batch[:,:,:,1] * recon_mask, recon_mask
            ), 1).transpose(2,1)
            
            loss_info = net.compute_unsupervised_loss(
              batch[:, 0, :,0],
              context_y,
              batch[:, 0, :,0],
              recon_context_y,
              logerr,
              weights,
              num_samples=k_iwae,
              qz_mean=qz_mean
            )
            
            individual_nlls.append(loss_info.loglik_per_ex.cpu().numpy())
            num_context_points = recon_mask.sum().item()
            individual_nlls.append(loss_info.loglik_per_ex.cpu().numpy())
            
            train_loss += loss_info.composite_loss.item()* num_context_points
            mse += loss_info.mse * num_context_points
            mae += loss_info.mae * num_context_points
            mean_mse += loss_info.mean_mse * num_context_points
            mean_mae += loss_info.mean_mae * num_context_points
            avg_loglik += loss_info.mogloglik * num_context_points
            
            mses.append(loss_info.mse)
            indy_nlls.append(loss_info.mogloglik)
            loss_info.mse
            
            train_n += num_context_points
    print(
        'nll: {:.4f}, mse: {:.4f}, mae: {:.4f}, '
        'mean_mse: {:.4f}, mean_mae: {:.4f}'.format(
            - avg_loglik / train_n,
            mse / train_n,
            mae / train_n,
            mean_mse / train_n,
            mean_mae / train_n
        )
    )
    return -avg_loglik / train_n, mse / train_n, np.concatenate(individual_nlls, axis=1)[0], indy_nlls, mses
    
    
    
    ## get qzs from folder, ensuring sure they're ordered the same as the loaded dataset
# qzs = np.zeros((l,num_ref_points*latent_dim,2))
# folder = 'qzs_all'
# for obj in os.listdir(folder):
#     qz = pd.read_csv(os.path.join(folder, obj,f'{obj}_qz.dat'), sep=' ',header=None).to_numpy()
#     i = test.valid_files_df.index.get_loc(obj)
#     qzs[i] = qz


# def set_intrinsic_var(self):
#     """
#     setting the normalized excess variance, as per the definition in Sánchez-Sáez et al. 2021
#     """
#     intrinsic_vars = np.zeros((len(self.dataset),len(self.bands)))
#     for i, object_lcs in enumerate(self.dataset):
#         for j, lc in enumerate(object_lcs):
#             dev_from_mean = (lc[:,1] - np.mean(lc[:,1]))
#             avg_sq_dev_from_mean = np.matmul(dev_from_mean, dev_from_mean) * (1 / (len(lc) - 1))
#             avg_sq_err = np.matmul(lc[:,2], lc[:,2]) / len(lc)
#             intrinsic_var = avg_sq_dev_from_mean - avg_sq_err
#             intrinsic_vars[i,j] = intrinsic_var
#     self.intrinsic_var = intrinsic_vars

        
##  better software principles w/ dataset class,
    # what does a dataset have? normalizing functions, 

# how we're gonna do synthetic data
    ## do the convolution thing once we have one band's carma fit to get others, then manually add lags
    
## could test the effects of length / num points on anomaly detection w/ syntehtic data, anomalous fits
# for dataset balancing, its probably more about number of observed points
# for training, its more about light cures being in the same time ranges 


## 12/27
    ## mcmc sample errors on the fly during training, error bar code 

# weird decisions to make (for talkinig purposes)
    # normlaize across training or individually? 
    # how to 'chop' by lenght or num points, min points?
    # injecting domain-specific knowledge
        # >1 mag err, keep obejcts between 13-20 mag, objects w/ certain intrinsitc var
    # formatting, setting masks
    
    # hyper-parameter things
    
    # how to use error bars on input data? 
        
        
    # lr scheduling? 
    # visualize kl annealing? wait_until_kl_inc is a hyperparam! 

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import argparse
from random import SystemRandom
from argparse import Namespace
from model import load_network
import utils
import optuna
from optuna.trial import TrialState
import sys
import logging
import warnings
from torch.optim import lr_scheduler

warnings.simplefilter('ignore', np.RankWarning) # set warning for polynomial fitting
LCS = utils.get_data('../../datasets/ZTF_g_test', start_col=1)

### trials is first command line arg, niters per trial is second  ####

def define_model_args(trial):
    args = Namespace(
        niters=int(sys.argv[2]),  ### num iters per trial 
        enc_num_heads=trial.suggest_categorical("enc_num_heads",[1,2,4,8,16,32]),
        embed_time = trial.suggest_categorical("embed_time",[32,64,128,256]),
        width=trial.suggest_categorical('width', [128,256,512,1024]),
        num_ref_points=trial.suggest_categorical('num_ref_points', [8,16,24,32,48]),
        rec_hidden=trial.suggest_categorical("rec_hidden",[32,64,128,256]),
        latent_dim= trial.suggest_categorical("latent_dim",[32,64,128,256]),
        dropout =0.0,#trial.suggest_float("dropout", 0.0,0.5,step=0.1),
        lr=0.0003,
        frac = 0.5,
        mixing='concat',
        mse_weight=5,#trial.suggest_int("mse_weight",1,20),
        data_folder = 'ZTF_gband_test',
        batch_size = 2,
        patience = 10000,
        scheduler = False,
        warmup = 4000,#trial.suggest_int('warmup', 3000,5000),
        k_iwae=1,
        kl_annealing=True,
        kl_zero=False, 
        net='hetvae', 
        norm=True, 
        save=False, 
        seed=0, 
        std=0.1, 
        device='mps',
        checkpoint = '',
        save_at = 10000000, 
        inc_errors = False,
        nodet=False,
        print_at=1
    ) 
    return args


def train(trial, args, lcs):
    experiment_id = int(SystemRandom().random() * 10000000)
    #print(args, experiment_id)
    ##################################
    seed = args.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    ##################################
    device = torch.device(args.device)
    
    data_obj = lcs.data_obj
    N_union_tp = trial.suggest_categorical('N_union_tp', [500,1000,1500,2000,2500,3000,3500,4000,4500,5000])
    lcs.set_union_tp(uniform=True, n=N_union_tp)
    train_loader = data_obj["train_loader"]
    test_loader = data_obj["test_loader"]
    val_loader = data_obj["valid_loader"]
    dim = data_obj["input_dim"]
    union_tp = torch.tensor(lcs.union_tp)
    
    net = load_network(args, dim, union_tp)
    params = list(net.parameters())
    optimizer = optim.Adam(params, lr=args.lr)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)
    epoch = 1
    loss = 1000000
    train_losses = []
    test_losses = []
    val_losses = []
    lrs = []
    
    model_size = utils.count_parameters(net) 
    print(f'{model_size=}')
    ############### have patience ##########
    best_loss = loss
    patience_counter = 0
    ######################################## 
    if args.kl_annealing:
        kl_coefs = utils.frange_cycle_linear(6000, n_cycle=16)
    ##################
    for itr in range(epoch, epoch+args.niters):
        print(f'{itr},', end='', flush=True)
    
        train_loss = 0
        train_n = 0
        avg_loglik, avg_wloglik, avg_kl, mse, wmse, mae = 0, 0, 0, 0, 0, 0 
        if args.kl_annealing:
            kl_coef = kl_coefs[itr] # need global number of epochs to continue on based on schedule
        elif args.kl_zero:
            kl_coef = 0
        else:
            kl_coef = 1
        ###################################################################  
        for train_batch in train_loader:
            batch_len = train_batch.shape[0]
            
            ### including obs error ##############################
            errorbars = torch.swapaxes(train_batch[:,:,:,2], 2,1)
            weights = errorbars.clone()
            weights[weights!=0] = 1 / weights[weights!=0]
            errorbars[errorbars!=0] = torch.log(errorbars[errorbars!=0]**2)
            logerr = errorbars.to(device) # log variance on observations 
            weights = weights.to(device)
            
            ############################################################
            subsampled_mask = utils.make_masks(train_batch, frac=args.frac)
            train_batch = train_batch.to(device)
            subsampled_mask = subsampled_mask.to(device)
            recon_mask = torch.logical_xor(subsampled_mask, train_batch[:,:,:,1])
            context_y = torch.cat((
              train_batch[:,:,:,1] * subsampled_mask, subsampled_mask), 1).transpose(2,1)
            recon_context_y = torch.cat((
                train_batch[:,:,:,1] * recon_mask, recon_mask), 1).transpose(2,1)
            #############################################################
            loss_info = net.compute_unsupervised_loss(
                train_batch[:, 0, :,0],
                context_y,
                train_batch[:, 0, :,0],
                recon_context_y,
                logerr,
                weights,
                num_samples=args.k_iwae,
                beta=kl_coef,
            )
            optimizer.zero_grad()
            
            if args.inc_errors:
                loss_info.weighted_comp_loss.backward()
            else:
                loss_info.composite_loss.backward()
                
            optimizer.step()
            
            ########### train loss ##################################
            train_loss += loss_info.composite_loss.item() * batch_len
            avg_loglik += loss_info.loglik * batch_len
            avg_wloglik += loss_info.wloglik * batch_len
            avg_kl += loss_info.kl * batch_len
            mse += loss_info.mse * batch_len
            wmse += loss_info.wmse * batch_len
            mae += loss_info.mae * batch_len
            train_n += batch_len
            #########################################################
        
    
        ####### nan, stop training #############
        if np.isnan(train_loss / train_n):
            print('nan in loss,,,,,,,,, stopping')
            break
        
        #### validation loss
        val_nll, val_mse, val_indiv_nlls = utils.evaluate_hetvae(
            net,
            dim,
            val_loader, 
            0.5,
            device=args.device
        )
        
        lrs.append(optimizer.param_groups[0]['lr'])
        if scheduler:
            scheduler.step(val_nll) 
        
        ######## print train / valid losses ########
        if itr % args.print_at == 0:
            print(
                '\tIter: {}, train loss: {:.4f}, avg nll: {:.4f}, avg wnll: {:.4f}, avg kl: {:.4f}, '
                'mse: {:.6f}, wmse: {:.6f}, mae: {:.6f}, val nll: {:.4f}, val mse {:.4f}'.format(
                    itr,
                    train_loss / train_n,
                    -avg_loglik / train_n,
                    -avg_wloglik / train_n,
                    avg_kl / train_n,
                    mse / train_n,
                    wmse / train_n,
                    mae / train_n,
                    val_nll,
                    val_mse))
                
        ####### test loss every 10 itrs, save losses too #############
        if itr % 10 == 0:
#             test_nll, test_mse, indiv_nlls = utils.evaluate_hetvae(
#                 net,
#                 dim,
#                 test_loader, 
#                 0.5,
#                 device=args.device
#                 )
            
            train_losses.append([(-avg_loglik / train_n).item(),(mse / train_n).item(),(avg_kl / train_n).item()])
#             test_losses.append([test_nll.item(),test_mse.item()])
            val_losses.append([val_nll.item(),val_mse.item()])
#             print('test nll: {:.4f}, test mse: {:.4f}'.format(test_nll,test_mse))
            

        ###### optuna stuff #######################        
        trial.report(val_nll, itr)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    return val_nll



def objective(trial):
    args = define_model_args(trial)
    loss = train(trial,args, LCS)
    return loss


def main():
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=int(sys.argv[1]), timeout=10000)
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    

if __name__ == '__main__':
    main()
        


In [ ]:
import glob
import utils
import sys

#### plotting forecasts, and getting forecast losses

for i, obj_folder in enumerate(glob.glob('./datasets/obj_dirs/*')):
    obj_name = obj_folder.split('/')[-1].split('_')[1]
    print(f'{i},', end='', flush=True)
    print(obj_folder)
    
    try:
        lcs = utils.get_data(obj_folder, sep=',', start_col=1, batch_size=1, min_length=40, \
                             n_union_tp=3500, num_resamples=111, shuffle=False, chop=False)
        train = lcs.data_obj['train_loader']

    except Exception as e:
        print(e)
        continue
        
    net, optimizer, args, epoch, loss = utils.load_checkpoint('./ZTFgr_small0.7880523800849915.h5', lcs.data_obj, device='cuda')
    examples, z, recons = utils.predict(train, net, device='cuda', subsample=False, target_x=None, forecast=True)
    utils.save_recon(examples, recons,z,obj_name, bands=lcs.bands, save_folder='forecast_plots')

    train_nll, mse, individual_nlls = utils.evaluate_hetvae(
                net,
                2,
                train,
                0.5,
                k_iwae=2,
                device='cuda',
                forecast=True
                )

    with open('forecast_losses.txt', 'a') as f:
        f.write(f'{obj_name},{train_nll},{mse}\n')
        
        
    
    
    

In [ ]:
import glob
import utils
import sys

#### plotting forecasts, and getting forecast losses

for i, obj_folder in enumerate(glob.glob('./datasets/obj_dirs/*')):
    obj_name = obj_folder.split('/')[-1].split('_')[1]
    print(f'{i},', end='', flush=True)
    print(obj_folder)
    
    try:
        lcs = utils.get_data(obj_folder, sep=',', start_col=1, batch_size=1, min_length=40,\
                             n_union_tp=3500, num_resamples=111, shuffle=False, chop=True, extend=5000)
        
        train = lcs.data_obj['train_loader']
        lcs.set_target_x(5001)

    except Exception as e:
        print(e)
        continue
        
    net, optimizer, args, epoch, loss = utils.load_checkpoint('./ZTFgr_small0.7880523800849915.h5', \
                                                              lcs.data_obj, device='cuda')
    
    examples, z, recons = utils.predict(train, net, device='cuda', subsample=False, \
                                        target_x=lcs.target_x, forecast=False)
    
    utils.save_recon(examples, recons,z,obj_name, bands=lcs.bands, save_folder='interpolate_plots')


    train_nll, mse, individual_nlls = utils.evaluate_hetvae(
                net,
                2,
                train,
                0.5,
                k_iwae=2,
                device='cuda',
                forecast=False
                )

    with open('losses.txt', 'a') as f:
        f.write(f'{obj_name},{train_nll},{mse}\n')
        
    
    

In [ ]:
import sys
import os
import shutil
import glob

#index = int(sys.argv[1])
lcs = os.listdir('./ZTFgr_small/g/')
obj_names = [lc.split('_')[0] for lc in lcs]
print(len(obj_names))
for obj_name in obj_names:
    dir_name = os.path.join('obj_dirs', f'ZTFgr_{obj_name}')
    os.mkdir(dir_name)
    g = f'{dir_name}/g'
    r = f'{dir_name}/r'
    os.mkdir(g)
    os.mkdir(r)
    g_file = glob.glob(f'ZTFgr_small/g/{obj_name}*')[0]
    r_file = glob.glob(f'ZTFgr_small/r/{obj_name}*')[0]
    print(dir_name)
    shutil.copy(g_file,g)
    shutil.copy(r_file,r)

In [ ]:
#!/bin/bash
#SBATCH -N 1
#SBATCH -c 1
#SBATCH --gres=gpu:1
#SBATCH -p res-gpu-large
#SBATCH --qos=long-high-prio
#SBATCH --job-name=hetvae
#SBATCH --output="test_loss-%j.out"
#SBATCH -t 10:00:00

source /etc/profile
source activate $HOME/miniconda/envs/hetvae

python3 $HOME/astr/hetast/src/forecast_script.py


In [ ]:
 def decode(net,qzs,disc_path,target_x, n_samples=10, device='mps', batch_size=1):
    
#     #input qzs shape (n_examples, 2, 16, 64)
#     #input disc path shape (268, 16, 64)
    
#     print(qzs.shape)
#     print(disc_path.shape)
    
#     z = (np.random.randn(n_samples, qzs.shape[0],qzs.shape[2], qzs.shape[3]) \
#          * qzs[:,0,:,:] + qzs[:,1,:,:]) # sample z
#     z = torch.from_numpy(z)
    
#     disc_path = torch.from_numpy(disc_path)
#     disc_path = disc_path.unsqueeze(0).repeat_interleave(n_samples,dim=0)
    
#     z = torch.cat((z, disc_path), -1) # 10,234,16,128, 
#     z = torch.swapaxes(z,0,1)
#     z = z.type(torch.Tensor)
    
#     pred_mean, pred_std = [], []
#     target_tp = []
#     dl = torch.utils.data.DataLoader(z, batch_size=batch_size, shuffle=False)
    
#     with torch.no_grad():
#         for i, batch in enumerate(dl):
#             batch = batch.to(device)
#             tx = torch.tensor(target_x[i*batch_size:(i*batch_size + batch_size)])[:,0]
#             tx = tx.to(device)
#             # zbatch shape should be 10,1,16,128
#             px = net.decode(batch.swapaxes(0,1), tx) 
#             pred_mean.append(px.mean.cpu().numpy())
#             pred_std.append(torch.exp(0.5 * px.logvar).cpu().numpy())
#             target_tp.append(tx.cpu().numpy())

#     pred_mean = np.concatenate(pred_mean, axis=1)
#     pred_std = np.concatenate(pred_std, axis=1)
#     target_tp = np.concatenate(target_tp, axis=0)
#     return pred_mean,pred_std,target_tp


In [ ]:

def save_recons(recon_info, obj_names, bands=['r','i','g'], save_folder='reconstructions'):
    examples = recon_info['examples']
    recons = recon_info['recons']
    zs = recon_info['zs']
    target_tp = recons['target_tp']
    pred_mean = recons['pred_mean']
    pred_std = recons['pred_std']
    target = examples['target']
    inputs = examples['inputs']
    tp = examples['tp']
    if not os.path.isdir(save_folder): os.mkdir(save_folder)     
    for i in range(len(inputs)):
        obj_folder = os.path.join(save_folder, obj_names[i])
        if not os.path.isdir(obj_folder): os.mkdir(obj_folder)
        pred_t = target_tp[i].nonzero()[0] # same for all bands? 
        t = target_tp[:1,pred_t]
        for j,band in enumerate(bands):
            mean_mag = pred_mean[i,pred_t,j][np.newaxis]
            mag_std = pred_std[i,pred_t,j][np.newaxis]
            lc = np.concatenate((t, mean_mag, mag_std), axis=0).T
            save_file = os.path.join(obj_folder,f'{obj_names[i]}_{band}.dat')
            np.savetxt(save_file, lc)
            print(f'{lc.shape=} and {band=} saved to {save_file}')
        z_save_file = os.path.join(save_folder,obj_names[i],f'{obj_names[i]}_qz.dat')
        zmu = zs['qz_mean'][i].flatten()
        zstd = zs['qz_std'][i].flatten()
        z = np.concatenate((zmu[np.newaxis], zstd[np.newaxis]), axis=0).T
        np.savetxt(z_save_file,z)
        print(f'{z.shape=} for {obj_names[i]=} saved to {z_save_file}')
        

In [ ]:
def save_qzs(qzs, obj_names, bands=['r','i','g'], save_folder='qzs'):
    if not os.path.isdir(save_folder): os.mkdir(save_folder)     
    for i in range(len(qzs)):
        obj_folder = os.path.join(save_folder, obj_names[i])
        if not os.path.isdir(obj_folder): os.mkdir(obj_folder)
        qz_save_file = os.path.join(save_folder,obj_names[i],f'{obj_names[i]}_qz.dat')
        np.savetxt(qz_save_file,qzs[i])
        print(f'{qzs[i].shape=} for {obj_names[i]=} saved to {qz_save_file}') 
    

In [ ]:
print('Intra-example formatting:\n')

print('light curve g\n')
print('  t    mag  magerr')
print(np.array([[0,4.3,0.09],
        [1.2,5.2,0.08]]))
print('\n')
print('light curve r\n')
print(np.array([[0,3.2,0.12],
        [2.33,4.1,0.03]]))
print('\n')
print('becomes:\n')
print('light curve g\n')
print(np.array([[0,4.3,0.09],
        [1.2,5.2,0.08],
        [2.33,0,0]]))
print('\n')
print('light curve r\n')
print(np.array([[0,3.2,0.12],
                [1.2,0.0,0.00],[2.33,4.1,0.03]]))
print('\nif 5 is the length of the longest \nintra-example union-time array, becomes:\n')
print('light curve g\n')
print(np.array([[0,4.3,0.09],
        [1.2,5.2,0.08],
        [2.33,0,0],[0,0,0],[0,0,0]]))
print('\n')
print('light curve r\n')
print(np.array([[0,3.2,0.12],
                [1.2,0.0,0.00],[2.33,4.1,0.03],[0,0,0],[0,0,0]]))


In [ ]:
def save_synth_data(base_folder='/Users/mattlowery/Desktop/code/astro/hetast/src/test_data', save_folder='synth_test_data', seed = 0, kernel='drw',duration=730, n=180, uniform=False):
    """
    This function creates and loads a synthetic dataset relative to a given kernel (drw or dho), 
    distributing the kernel params relative to a real dataset  
    
    parameters:
        folder             (str)      --> synthetic data will be fit to the light curves in this folder to get an honest distribution of params
         ----- optional -----
        seed               (int)      --> random seed, this matters to keep the shuffles consistent
        batch_size         (int)      --> for the network, usually a multiple of 2
        kernel             (str)      --> 'drw' (dampled random walk), a carma(1) process; 'dho' (damped harmonic oscillator), a carma(2,1) process
        n
        duration 
        
    drw_kernel params --> 'tau' is decorelation timescale, 'amp' is the amplitude
    dho_kernel params --> a1 = 2 * Xi * w0
                       a2 = w0 ** 2
                       b0 = sigma
                       b1 = tau * b0
                       wherein Xi is the damping ratio, w0 is the natural oscillation frequency, sigma is the amplitude
                       of the short term perturbing white noise, tau is the characteristic timescale of the perturbation process
    returns:
        a dictionary of torch dataloaders with data formatted as necessary for network training 
        , as well as the dimension and union of all the time points
    """
    synth_band_name = 'simband1'
    
    np.random.seed(seed)
    if not os.path.isdir(base_folder):
        raise Exception(f"{base_folder} is not a directory")
        
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
        os.mkdir(os.path.join(save_folder,synth_band_name))   
        
    lcs = DataSet(name=base_folder, min_length=50, sep=',', start_col=1)
    band_folders = os.listdir(base_folder)
    for band_folder in band_folders:
        band = band_folder.lower()
        lcs.add_band(band, os.path.join(base_folder, band_folder))
        
    lcs.filter()         
    lcs.prune_outliers()
    lcs.set_carma_fits(kernel=kernel)
    lcs.set_snr()
    
    synth_lcs = []
    for i, params in enumerate(lcs.carma_fits):
        if np.isnan(params[0]):
            continue
        if kernel == 'drw':
            term = DRW_term(*np.log(params))
        else:
            term = DHO_term(*np.log(params))
                
        # kernel, snr, duration (days), n      
        if uniform == True:
            lc = np.array(gpSimFull(term,lcs.snr[i,0], duration, n)).transpose(1,0)
        else: 
            lc = np.array(gpSimRand(term,lcs.snr[i,0], duration, n)).transpose(1,0)   
        fpath = f'{os.path.join(save_folder, synth_band_name,str(i))}_.csv'
        np.savetxt(fpath,lc, delimiter=',', header=' , , ', comments='')

In [ ]:
    def set_carma_fits(self, kernel='drw'):
        carma_fits = []
        for i, object_lcs in enumerate(self.dataset):
            print(i, end='')
            lc = object_lcs[0]
            if kernel == 'drw':
                fit = drw_fit(lc[:,0], lc[:,1], lc[:,2])
            else:
                fit = dho_fit(lc[:,0], lc[:,1], lc[:,2])   
            carma_fits.append(fit)          
        self.carma_fits = carma_fits

In [ ]:
        ranges = [np.ptp(lc[:,0]) for object_lcs in self.dataset for lc in object_lcs]
        bins = np.arange(max(ranges), step=50)
        range1 = [np.ptp(object_lcs[0][:,0]) for object_lcs in self.dataset]
        range2 = [np.ptp(object_lcs[1][:,0]) for object_lcs in self.dataset]
        range3 = [np.ptp(object_lcs[2][:,0]) for object_lcs in self.dataset]
        plt.hist(range1,alpha=0.45,label='r',bins=bins)
        plt.hist(range3,alpha=0.35,label='g',bins=bins)
        plt.hist(range2,alpha=0.45,label='i',bins=bins)
        
        plt.title('Distribution of Light Curve Ranges')
        plt.legend(loc="upper left")
        plt.xlabel('range (days)')
        plt.savefig('ranges')

In [ ]:
#source ~/miniconda/bin/activate

In [114]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:
%autoreload

In [ ]:
 #might need an interpolation turn off (just predict based on ), look at predict function one more time